# SparkR ile Lineer Modelleme

[R ile Apache Spark a giriş, Hakan Sarıbıyık](https://github.com/vezir/spark-r-notebooks)

Bu notebookda, [SparkR](http://spark.apache.org/docs/latest/sparkr.html) kullanımı yolu ile *ABD - California da yol kazalarında yaralanmalar 2002-2010* [Road Traffic Injuries 2002-2010](http://www.healthdata.gov/dataset/road-traffic-injuries-2002-2010) veri setini kullanarak kazalarda yaralanan kişilerin sayısını tahminlemek için lineer bir model geliştirmeye çalışacağız. Bunu SparkR da yapmamızın sebebi SparkR ın ölçeklenebilir mimarisinin büyük veri ile çalışmaya konusunda verdiği olanakları kullanmaktır. Bu sayede, veri miktarı R da çalışmayı zor hatta imkansız kıldığı durumlarda dahi modelleme yapabilmek mümkün olmaktadır.

R da olan fonksiyonların benzerlerini SparkR da olduğu ölçüde kullacağız, bazı durumlarda verinin boyutu izin verdiği durumlarda doğrudan R da iş yapacağız.



## SparkSQL Context i oluşturmak ve verimizi yükleme

Veri setimiz, *ABD - California da yol kazalarında yaralanmaları 2002-2010* [Road Traffic Injuries 2002-2010](http://www.healthdata.gov/dataset/road-traffic-injuries-2002-2010) ile ilgili verileri içeriyor.

Bu veri setinde Kaliforniya'da yaşayan kişi ve mil başına olan trafik kazalarının yaya, otomobil, motorsiklet gibi kategorilerdeki istatistikleri, Kaliforniya'nın alt bölgeleri bazında verilmektedir. Veriyi doğrudan incelemek isterseniz [analiz için hazırlanmış sayfadan](https://cdph.data.ca.gov/Environment/Road-Traffic-Injuries-2002-2010/xmwz-xvsf) faydalanabilirsiniz. Var olan alanların neler olduğu ile ilgili bir [excel doküman](https://cdph.data.ca.gov/api/views/xmwz-xvsf/files/vFZ2-VvAdPb_6aOkATlLb19r3PpHHYGEgns1EH3kAQs?download=true&filename=RoadTrafficInjuries_DD.xlsx) da mevcuttur.

## SparkSQL context i yaratmak

Bu ve sonraki notebooklarda veriyi dataFrame aktarmak için öncelikle bir SparkSQL context e ihtiyacımız olacak. Ayrıca, SPARK_HOME gibi temel değişkenlere uygun değerleri atamamız da gerekiyor.

In [1]:
# Spark ın kurulduğu dizin
Sys.setenv(SPARK_HOME="/usr/local/spark")
# SparkR ın kurulduğu dizinden yüklenmesi için gerekiyor
.libPaths(c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib"), .libPaths()))

SparkR kütüphanesini yükleyelim.

In [2]:
library(SparkR)


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var

The following objects are masked from ‘package:base’:

    colnames, colnames<-, intersect, rank, rbind, sample, subset,
    summary, table, transform



Spark ı kullanabilmemiz için bir SparkContext te ihtiyacımız var. Bunu Spark ın [sayfasında](http://spark.apache.org/docs/latest/sparkr.html#starting-up-sparkcontext-sqlcontext) anlatıldığı şekilde yapacak olursak sparkR.init komutunu kullanmamız gerekiyor. Burada master olarak Spark ın bulundugu makinanın IP sini yada lokalde ise *local* kelimesini kullanıyoruz.

In [3]:
sc <- sparkR.init(master="local", sparkPackages="com.databricks:spark-csv_2.11:1.2.0")

Launching java with spark-submit command /usr/local/spark/bin/spark-submit  --packages com.databricks:spark-csv_2.11:1.2.0 sparkr-shell /tmp/RtmpqWcDyD/backend_port94e43a60ed3 


Bu şekilde emrimizi bekleyen bir spark elde ettik. sparkPackages a koyduğumuz paket csv formatındaki dosyaları okumak için kullanılan bir paket. Artık dataFrame oluşturmak için gereken sparkSQL context i oluşturabiliriz. Çalıştırdığımız işlerin detay takibini standart olarak http://10.0.2.15:4040 adresinden browser yardımı ile yapabiliriz. Jupyter notebook un loglarından sizinkini görebilirisiniz. Artık sqlContext e geçebiliriz.

In [4]:
sqlContext <- sparkRSQL.init(sc)

## Veriyi yükleyelim
SQLcontext imizi oluşturduğumuza göre verimizi DataFrame yükleyerek detaylı analize başlayabiliriz. Önce veriyi yükleyelim. Bunun için [ilk notebook](https://github.com/vezir/spark-r-notebooks/blob/master/notebooks/1-baslangic/baslangic.ipynb)da yaptığımızı yapacağız ama web den yüklemek yerine orada bahsettiğimiz şekilde lokalde nereye kopyaladıksak oradan alacağız.

In [5]:
traffic_injuries_file_path <- file.path('','home','dsuser','shared','Road_Traffic_Injuries.txt')

In [6]:
print(traffic_injuries_file_path)

[1] "/home/dsuser/shared/Road_Traffic_Injuries.txt"


In [23]:
system.time(
    traffic_injuries_df <- read.df(sqlContext, 
                        paste('file:', traffic_injuries_file_path, sep=''), 
                        header='true', 
                        source = "com.databricks.spark.csv", 
                        inferSchema='true')
)

   user  system elapsed 
  0.040   0.004  11.174 

In [24]:
traffic_injuries_df$region_code <- cast(traffic_injuries_df$region_code, "string")
nrow(traffic_injuries_df)

[1] 494226

## Verinin modelleme için hazırlanması

Modellemede regresyon kullanacağımız için, boş değerleri içeren gözlemleri ne yapacağımıza karar vermemiz gerekiyor. Boş değerleri olan kayıda denk gelen satırları ya tamamen kaldıracağız, ya da bu boş değerlerin yerlerine veri setine uygun gördüğümüz değerlerle dolduracağız. Bu işleme veri madenciliği literatüründe *imputation* denmektedir. Bu boş değerle o kolonun ortalama değerlerini, min veya maks değerlerini koyabiliriz. Bu tamamen o kolonda tutulan verinin özelliğine bağlıdır. 

Biz burada boş değerleri tahminlemede kullanmaya karar verdiğimiz değişkenleri ve beklenen değeri tutan *injuries* değişkeninde boş olan satırları tamamen çıkararak devam edeceğiz.


Gördüğümüz kadarı ile *reportyear* 2002-2010 arasında yıllık olan verimiz, ayrıca 2002-2004, 2005-2007, 2008-2010 şeklinde 3 yıllık ortalamalar şeklinde verilmiş, bir de 2006-2010 yılllarında 5 yıllık bir ortalama da verilmiş. Bunu [Veri sözlüğümüz](https://cdph.data.ca.gov/api/views/xmwz-xvsf/files/vFZ2-VvAdPb_6aOkATlLb19r3PpHHYGEgns1EH3kAQs?download=true&filename=RoadTrafficInjuries_DD.xlsx) den de kontrol ettiğimizde tanımla uyumlu olduğunu görüyoruz.

"2002 to 2010; 2002-2004, 2005-2007, and 2008-2010 3 year averages; 2006-2010 5 year average"

Veriyi analiz ederken bunu dikkate almamız gerekiyor, yoksa sonuçlarımız bizi anlamsız çıkarımlara götürür. Biz yapacağımız çalışmada yıllık veriler dışında kalan ortalama verileri filtreleyelim. 

SparkR da R daki %in% operatörü olmadığı için şöyle yapıyoruz.

In [80]:
traffic_injuries_2002_2010_df <- filter(traffic_injuries_df, 
                                        traffic_injuries_df$reportyear=="2002" | 
                                        traffic_injuries_df$reportyear=="2003" |
                                        traffic_injuries_df$reportyear=="2004" |
                                        traffic_injuries_df$reportyear=="2005" |
                                        traffic_injuries_df$reportyear=="2006" |
                                        traffic_injuries_df$reportyear=="2007" |
                                        traffic_injuries_df$reportyear=="2008" |
                                        traffic_injuries_df$reportyear=="2009" |
                                        traffic_injuries_df$reportyear=="2010"
                                     )
nrows <- nrow(traffic_injuries_2002_2010_df)
nrows

[1] 274561

In [129]:
traffic_injuries_filtered_df <- filter(
    traffic_injuries_2002_2010_df, 
    isNotNull(traffic_injuries_2002_2010_df$injuries) 
    & isNotNull(traffic_injuries_2002_2010_df$geotypevalue)
    & isNotNull(traffic_injuries_2002_2010_df$mode)
    & isNotNull(traffic_injuries_2002_2010_df$totalpop)
    & isNotNull(traffic_injuries_2002_2010_df$geotype)
)
nrows <- nrow(traffic_injuries_filtered_df)
nrows

[1] 37359

In [130]:
traffic_injuries_filtered_df <- 
  traffic_injuries_filtered_df[traffic_injuries_filtered_df$geotype =='CO',]
head(traffic_injuries_filtered_df)

,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,ellip.h,avmttotal,avmtrate,LL95CI_avmtrate,UL95CI_avmtrate,avmtrate_se,avmtrate_rse,CA_decile_avmt,CA_RR_avmtrate,groupquarters,version
1,753,Annual number of fatal and severe road traffic injuries per population and per miles traveled by transport mode,2002,9,Total,CO,6001,Alameda,Alameda,6001,⋯,NA,NA,NA,NA,NA,NA,,NA,29643,10/10/2014 12:00:00 AM
2,753,Annual number of fatal and severe road traffic injuries per population and per miles traveled by transport mode,2002,9,Total,CO,6001,Alameda,Alameda,6001,⋯,NA,NA,NA,NA,NA,NA,,NA,29643,10/10/2014 12:00:00 AM
3,753,Annual number of fatal and severe road traffic injuries per population and per miles traveled by transport mode,2002,9,Total,CO,6001,Alameda,Alameda,6001,⋯,NA,NA,NA,NA,NA,NA,,NA,29643,10/10/2014 12:00:00 AM
4,753,Annual number of fatal and severe road traffic injuries per population and per miles traveled by transport mode,2002,9,Total,CO,6001,Alameda,Alameda,6001,⋯,NA,NA,NA,NA,NA,NA,,NA,29643,10/10/2014 12:00:00 AM
5,753,Annual number of fatal and severe road traffic injuries per population and per miles traveled by transport mode,2002,9,Total,CO,6001,Alameda,Alameda,6001,⋯,NA,NA,NA,NA,NA,NA,,NA,29643,10/10/2014 12:00:00 AM
6,753,Annual number of fatal and severe road traffic injuries per population and per miles traveled by transport mode,2002,9,Total,CO,6001,Alameda,Alameda,6001,⋯,NA,NA,NA,NA,NA,NA,,NA,29643,10/10/2014 12:00:00 AM


Yılları ayrı değerlendirmek yerine modeli basit tutmak için ortalamaları almak genel bir yöntemdir. Ortalamalar değişik şekillerde alınabilir. Biz basit ortalama alacağız.

In [131]:
traffic_injuries_filtered_df <-
    agg(
        groupBy(traffic_injuries_filtered_df, "geotypevalue", "geoname",
                             "totalpop","region_name","mode", "severity"),
        avg_injuries=avg(traffic_injuries_filtered_df$injuries) 
    )
head(traffic_injuries_filtered_df)

,geotypevalue,geoname,totalpop,region_name,mode,severity,avg_injuries
1,6009,Calaveras,42212,Central/Southeast Sierra,Vehicles,Severe Injury,43
2,6017,El Dorado,164748,Sacramento Area,Vehicles,Severe Injury,93
3,6031,Kings,135131,San Joaquin Valley,Pedestrian,Killed,1
4,6047,Merced,222982,San Joaquin Valley,Motorcycle,Severe Injury,10
5,6047,Merced,222982,San Joaquin Valley,Truck,Killed,4
6,6057,Nevada,94663,Northeast Sierra,Bus,Severe Injury,1


## Veriyi çalışma ve test etme olarak ikiye ayırma

Modellemeyi yaparken modelleme için kullanacağımız veriyi iki gruba ayırma ve bu gruplardan birini modeli eğitmede kullanırken, diğerini modelin performansını ölçmek için kullanma genel bir yöntemdir. Bu yöntemin bir çok değişik versiyonları olmakla birlikte burada bunlara değinmeyeceğiz.

SparkR ın bu noktada gelişmesi gereken bir yönüne denk geliyoruz. SparkR da [SPARK-6836](https://issues.apache.org/jira/browse/SPARK-6836) ile kayıtlı bekleyen bir iş nedeni ile, ML kütüphanesinde [TrainValidationSplit](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.tuning.TrainValidationSplit) class ında olan veri setini train ve validation şeklinde ikiye ayırma ile ilgili fonksiyonaliteyi burada kullanamıyoruz. Bu JIRA maddesi çözülene değin, kendimizin çözmesi gereken veriyi ikiye ayırma problemi için her bir ID alanına ihtiyacımız var. 

Bu ID alanını eklemek SparkR da hayli problemli. Bunun için kendim bir çözüm bulamayınca bir google araması yaptım ama henüz çözüm üretilmeyen problemlerden olduğunu öğrendim. Hal böyle olunca, geriye iki çözüm kalıyor. Birincisi, siz siz olun verinizi SparkR a yüklerken böyle bir ihtiyacı düşünerek veriye en başta bir ID alanı eklenmiş şekilde DataFrame e yükleyin. İkincisi ise, elimizdeki veri seti uygunsa önce data.frame e çevirin ID alanını ekleyin, sonrada DataFrame e dönüştürün. Biz ilkini yapmadığımız için ikinci çözümü kullanacağız.

Ayrıca, modele girmesine karar verdiğimiz değişkenleri seçelim. Bunu yaparken bir önceki notebook da nedenini anlattığımız şekilde *geotypevalue* değişkenini *string* yapıp başına '0' getirelim.

In [133]:
traffic_injuries_filtered_d.f <- collect(traffic_injuries_filtered_df)
traffic_injuries_filtered_d.f$ID <- 1:nrow(traffic_injuries_filtered_d.f)
traffic_injuries_filtered_df <- createDataFrame(sqlContext, traffic_injuries_filtered_d.f)

traffic_injuries_filtered_df = 
  withColumn(traffic_injuries_filtered_df, "geotypevalue0", 
             format_string('0%s', cast(traffic_injuries_filtered_df$geotypevalue, "integer"))
)     

traffic_injuries_filtered_df <- select(traffic_injuries_filtered_df, "ID",
                                       "avg_injuries", "region_name",
                                       "geotypevalue0", "geoname",
                                       "totalpop",
                                       "mode", "severity"
                                       )
head(traffic_injuries_filtered_df) 


,ID,avg_injuries,region_name,geotypevalue0,geoname,totalpop,mode,severity
1,1,43,Central/Southeast Sierra,06009,Calaveras,42212,Vehicles,Severe Injury
2,2,93,Sacramento Area,06017,El Dorado,164748,Vehicles,Severe Injury
3,3,1,San Joaquin Valley,06031,Kings,135131,Pedestrian,Killed
4,4,10,San Joaquin Valley,06047,Merced,222982,Motorcycle,Severe Injury
5,5,4,San Joaquin Valley,06047,Merced,222982,Truck,Killed
6,6,1,Northeast Sierra,06057,Nevada,94663,Bus,Severe Injury


In [134]:
printSchema(traffic_injuries_filtered_df)

root
 |-- ID: integer (nullable = true)
 |-- avg_injuries: double (nullable = true)
 |-- region_name: string (nullable = true)
 |-- geotypevalue0: string (nullable = false)
 |-- geoname: string (nullable = true)
 |-- totalpop: double (nullable = true)
 |-- mode: string (nullable = true)
 |-- severity: string (nullable = true)


Artık *ID* alanımız olduğuna göre işe koyulabiliriz. Önce *sample* fonksiyonu ile veri setinden rastgele bir altküme alalım. Ne kadar olduğu size kalmış. Ben test %20, çalışma/eğitme %80 yapmayı düşünüyorum.

In [135]:
traffic_injuries_filtered_df_test <- sample(traffic_injuries_filtered_df,FALSE,0.2)
nrow(traffic_injuries_filtered_df_test)

[1] 1306

In [136]:
test_ids <- collect(select(traffic_injuries_filtered_df_test, "ID"))$ID

In [137]:
traffic_injuries_filtered_df$IS_TEST <- traffic_injuries_filtered_df$ID %in% test_ids

In [138]:
traffic_injuries_filtered_df_train <- subset(traffic_injuries_filtered_df, traffic_injuries_filtered_df$IS_TEST==FALSE)
nrow(traffic_injuries_filtered_df_train)

[1] 4942

Buradaki işlemler SparkR da *split* ile ilgili bir fonksiyonun aciliyetini gösteriyor. Yoksa büyük veri setleri ile bu işleri yapmak daha da sıkıntılı olacağı açık.

## Lineer bir modelin eğitilmesi
Bu iş için neler gerekiyor.
* R da da olan *glm* fonksiyonu
* Eğitim için gereken veri seti
* Modelin tipi (Gausyan, yada binom)

Kullanım R dakinden pek farklılık göstermiyor. Şimdi modelde kullanacağımız ve bir önceki notebook da detaylarını verdiğimiz yukarıda *select* ile bir dataFrame de topladığımız değişkenleri sıralayalım.

Coğrafik değişkenler

* *geotype* : Type of geographic unit
* *geotypevalue* : Value of geographic unit
* *geoname* : Name of geographic unit
* *region_name* : Name of region

Nüfus değişkenleri

* *totalpop* : (Denominator 1) Total population in the geographic area

Seyahat ile ilgili değişkenler

* *mode* : Mode of transportation of the victim

Kurbanların yaralanma/ölme durumları ile ilgili değişkenler

* *severity* : Severity of the victim's injuries

Tipleri kontrol edelim.

In [139]:
printSchema(traffic_injuries_filtered_df_train)

root
 |-- ID: integer (nullable = true)
 |-- avg_injuries: double (nullable = true)
 |-- region_name: string (nullable = true)
 |-- geotypevalue0: string (nullable = false)
 |-- geoname: string (nullable = true)
 |-- totalpop: double (nullable = true)
 |-- mode: string (nullable = true)
 |-- severity: string (nullable = true)
 |-- IS_TEST: boolean (nullable = true)


In [141]:
model <- glm(
    avg_injuries ~ geoname + totalpop + mode + severity, 
    data = traffic_injuries_filtered_df_train, 
    family = "gaussian")

In [142]:
summary(model, signif.stars=TRUE)

$devianceResiduals
 Min       Max     
 -557.0523 2403.766

$coefficients
                        Estimate      Std. Error   t value     Pr(>|t|)    
(Intercept)             -246.1828     22.22977     -11.07447   0           
geoname_San Diego       920.3487      172.5699     5.333194    1.008609e-07
geoname_Los Angeles     2710.7        559.6934     4.843186    1.31729e-06 
geoname_Riverside       662.6117      115.09       5.757338    9.068379e-09
geoname_Sacramento      452.5343      81.01553     5.585772    2.452942e-08
geoname_San Bernardino  680.2915      113.3043     6.004108    2.063281e-09
geoname_Orange          840.0219      170.7261     4.92029     8.926118e-07
geoname_San Francisco   258.3553      50.18363     5.148199    2.733402e-07
geoname_Tulare          186.5334      32.43447     5.751084    9.407975e-09
geoname_Contra Costa    318.6192      62.10036     5.130714    2.998433e-07
geoname_Fresno          336.8306      55.16866     6.10547     1.104447e-09
geoname_Kern            311.6658      49.56643     6.287841    3.500247e-10
geoname_San Mateo       231.2882      46.07013     5.02035     5.341476e-07
geoname_San Joaquin     258.9825      43.42163     5.964366    2.629121e-09
geoname_Merced          126.6235      26.35106     4.805253    1.591962e-06
geoname_Stanislaus      215.4274      36.21253     5.948976    2.886662e-09
geoname_Sonoma          191.8166      35.09662     5.465386    4.848962e-08
geoname_Placer          141.0354      28.88658     4.882384    1.081591e-06
geoname_Solano          162.7421      32.4675      5.01246     5.564102e-07
geoname_Santa Clara     501.8561      101.1429     4.961851    7.220043e-07
geoname_Monterey        175.246       32.48874     5.394054    7.214082e-08
geoname_Santa Barbara   176.9195      32.71089     5.40858     6.656068e-08
geoname_Alameda         453.0976      87.42685     5.18259     2.276573e-07
geoname_Butte           114.4838      25.86712     4.425843    9.816345e-06
geoname_Santa Cruz      116.547       27.14147     4.294057    1.788459e-05
geoname_Imperial        101.5984      24.63298     4.124488    3.777462e-05
geoname_Yolo            93.98411      25.31656     3.712357    0.0002076301
geoname_El Dorado       102.6458      24.93377     4.116737    3.906264e-05
geoname_Ventura         284.705       51.21975     5.558502    2.865862e-08
geoname_San Luis Obispo 122.1802      27.4056      4.45822     8.449835e-06
geoname_Shasta          99.90795      25.0474      3.988755    6.73945e-05 
geoname_Napa            77.77678      24.2426      3.208269    0.001344011 
geoname_Madera          90.1046       24.46855     3.682466    0.0002334901
geoname_Mendocino       75.38311      23.64025     3.188761    0.001437883 
geoname_Yuba            57.4515       23.48413     2.446397    0.01446429  
geoname_Marin           97.83751      27.13173     3.606018    0.0003140431
geoname_Humboldt        80.29657      24.33703     3.299358    0.0009760202
geoname_Tehama          60.75075      23.47044     2.588394    0.009671105 
geoname_Kings           89.53932      24.71728     3.62254     0.000294694 
geoname_Sutter          60.72474      23.87438     2.54351     0.0110051   
geoname_San Benito      43.98991      23.72038     1.85452     0.06372518  
geoname_Nevada          77.49792      24.31092     3.187782    0.001442751 
geoname_Amador          42.02994      23.80261     1.76577     0.07749705  
geoname_Tuolumne        48.35174      23.9312      2.020448    0.04339155  
geoname_Siskiyou        47.26819      23.93494     1.974861    0.04834023  
geoname_Lassen          37.76126      23.87724     1.581475    0.1138344   
geoname_Lake            50.66767      24.05749     2.106108    0.03524595  
geoname_Inyo            49.70115      23.89405     2.080063    0.03757183  
geoname_Del Norte       25.44305      23.99209     1.060477    0.2889803   
geoname_Trinity         20.17793      24.01427     0.8402475   0.4008108   
geoname_Plumas          23.92304      24.02896    

Aldığımız summary nin sonuçlarına baktığımızda son kolondaki değerlerin istatistiksel olarak dikkate değer olması için aranan şart 0.05 den küçük olmasının bazı satırlarda sağlanmadığını görüyoruz. Bu modelimizin ilgili değişken için iyi çalışmayacağının bir göstergesi. Burada kategorik değişkenlerde otomatik kolon oluşturup regresyon yapıldığı için özellikle alt bölgelerin bazılarında problem olduğunu görebiliyoruz. 

## Test verisi ile modelimizin doğruluğunun kontrolü

In [144]:
injuries_mean <- collect(agg(
    traffic_injuries_filtered_df_train, 
    AVG_INJURIES=mean(traffic_injuries_filtered_df_train$avg_injuries)
))$AVG_INJURIES

injuries_mean

[1] 65.9917

In [145]:
predictions <- predict(model, newData = traffic_injuries_filtered_df_test)


Tahminlerimizin gerçekle ne kadar tutarlı olduğunu ölçmek için [R2](https://en.wikipedia.org/wiki/Coefficient_of_determination) değerini hesaplayalım.

In [148]:
predictions <- transform(
    predictions, 
    S_res=(predictions$avg_injuries - predictions$prediction)**2, 
    S_tot=(predictions$avg_injuries - injuries_mean)**2)
head(select(predictions, "avg_injuries", "prediction", "S_res", "S_tot"))

,avg_injuries,prediction,S_res,S_tot
1,93,133.3833,1630.814,729.4481
2,69,95.90106,723.667,9.049846
3,52,51.09473,0.8195144,195.7678
4,2,5.588882,12.88007,4094.938
5,82,99.96816,322.8548,256.2655
6,57,108.4535,2647.466,80.85074


In [149]:
nrows_test <- nrow(traffic_injuries_filtered_df_test)
residuals <- collect(agg(
    predictions, 
    SS_res=sum(predictions$S_res),
    SS_tot=sum(predictions$S_tot)
))

In [150]:
residuals

,SS_res,SS_tot
1,34627887,58968645


In [152]:
R2 <- 1.0 - (residuals$SS_res/residuals$SS_tot)
R2

[1] 0.4127746

R2 nin 1 değeri alması mükemmel tahminlemeyi, 0 değeri alması ise lineer olmayan bir veriyi veya tamamen rasgele bir veri ile çalışıldığını gösterir. Bu durumda bizim model iyi görünmüyor.

## Sonuç

Bu durumda modelimizin yaralanma/ölme rakamlarının tahminlemesini iyi yapmasını istiyorsak üzerinde biraz daha çalışmamız gerekiyor.

Spark v1.6 nın R üzerinde makina öğrenmesi yapan modülü bazı kısıtlamalar içeriyor. Bu kısıtlardan bazıları;

* Model parametrelerini yorumlamak için yardımcı olan *summary* komutunun çıktısında görmeye alışık olduğumuz şeylerin örnek; dikkate değer yıldızlar özelliği eklenmesi.
* Modelin çıktısını kolayca değerlendirmek için R2 gibi fonksiyonların hazır sunulması.
* verinin test ve eğitim için kolayca bölünmesi için bir *split* fonksiyonu
* Lineer olmayan modeller eklenmesi.
* R motoru harici wrapper lardan kaynaklı hata mesajlarının anlaşılır yapılması.